This script add 
- mean encoding by ip,os,device,os, app,channel

compare to features1.ipynb

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline 
import gc,time

In [2]:
with pd.HDFStore('../input/raw_data.h5') as store:
    print(store.keys())
    test_df = store['test_df']
    train_df = store['train_df']

['/test_df', '/train_df']


In [3]:
debug = 0
if debug:
    nrows = 100000
    nchunk = 400000
    val_size = 25000
    frm = nrows - 75000
    to = frm + nchunk
    
    test_size = 50000
    test_df = test_df.iloc[:test_size,:]
else:
    nrows=184903891-1
    nchunk=40000000

    val_size=2500000
    frm=nrows-75000000
    to = frm + nchunk

In [4]:
gc.collect()

200

In [5]:
trn_df = train_df.iloc[frm:to,:]
len_trn = len(trn_df)
trn_df = trn_df.append(test_df)

# del train_df;test_df;gc.collect()

In [6]:
print('trn_df :{}'.format(trn_df.shape)) # 
# print('test_df:{}'.format(test_df.shape))
del test_df

trn_df :(58790469, 10)


# helper func

In [7]:
def encode_agg_feature(train_df, selcols, groupby, aggregator = 'nunique'):
    usecols = [e for e in selcols if e not in  groupby]    
    
    if aggregator == 'nunique':  
        colname= usecols[-1] + '_nunique_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].nunique().reset_index().\
            rename(columns = {
                usecols[-1] : usecols[-1] + '_nunique_' + '_'.join(groupby)
            })
        train_df = train_df.merge(gp, how='left', on=groupby)

    elif aggregator == 'cumcount':
        colname = usecols[-1] + '_cumcnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].cumcount()        
        train_df[colname] = gp.values
    
    elif aggregator == 'count':
        colname = usecols[-1] + '_cnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
        gp = train_df[selcols].groupby(groupby)[usecols].count().reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
        
    elif aggregator in ['var','mean']:
        agg=np.var if aggregator == 'var' else np.mean
        
        colname = usecols[-1] + '_'+ str(aggregator) +'_' + '_'.join(groupby)
        
        if colname in train_df.columns:
            train_df.drop(colname,axis=1, inplace=True)
            
        gp = train_df[selcols].groupby(groupby).agg(agg).reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
    
    return train_df.iloc[:,-1]

#### feat_base

- channel cnt by (ip,day,hour)
- channel cnt by (ip,app)
- channel nunique by ip
- device nunique by ip
- app nunique by ip

In [8]:
print('feature1 : number of unique "channel" by ip',end='\t')
f1 = encode_agg_feature(trn_df,groupby=['ip'], selcols=['ip','channel'],aggregator='nunique')
print(f1.name)

print('feature2 : cumcount of "app" by (ip,device,os)',end='\t')
f2 = encode_agg_feature(trn_df,selcols=['app','ip','device','os'], groupby=['ip','device','os'], aggregator='cumcount')
print(f2.name)

print('feature3 : number of unique "hour" by (ip, day)',end='\t')
f3 = encode_agg_feature(trn_df,selcols=['hour','ip','day'], groupby=['ip','day'], aggregator='nunique')
print(f3.name)

print('feature4 : number of unique "app" by ip', end='\t')
f4 = encode_agg_feature(trn_df,selcols=['app', 'ip'], groupby=['ip'])
print(f4.name)

print('feature5 : number of unique "app" by (ip,os)',end='\t')
f5 = encode_agg_feature(trn_df,selcols=['app', 'ip', 'os'], groupby=['ip','os'])
print(f5.name)

print('feature6 : number of unique "device" by ip',end='\t')
f6 = encode_agg_feature(trn_df,selcols=['device', 'ip'], groupby=['ip'])
print(f6.name)

print('feature7 : number of unique "app" by channel',end='\t')
f7 = encode_agg_feature(trn_df,selcols=['app', 'channel'], groupby=['channel'])
print(f7.name)

print('feature8 : cumcount of "os" by ip ',end='\t')
f8 = encode_agg_feature(trn_df,selcols=['os', 'ip'], groupby=['ip'], aggregator='cumcount')
print(f8.name)

print('feature9 : number of unique "app" by (ip,device,os)',end='\t')
f9 = encode_agg_feature(trn_df,selcols=['app', 'ip','device','os'], groupby=['ip','device','os'])
print(f9.name)

#### count #####
print('feature 10 : count by (ip,day,hour)',end='\t')
f10 = encode_agg_feature(trn_df,selcols=['ip','day','hour','channel'],groupby=['ip','day','hour'],aggregator='count')
print(f10.name)

print('feature 11 : count by (ip,app)', end='\t')
f11 = encode_agg_feature(trn_df,selcols=['ip','app','channel'],groupby=['ip','app'],aggregator='count')
print(f11.name)

print('feature 12 : count by (ip,app,os)',end='\t')
f12 = encode_agg_feature(trn_df,selcols=['ip','app','channel','os'],groupby=['ip','app','os'],aggregator='count')
print(f12.name)

### mean,var ####
print('feature 13: var of hour by (ip,day,channel)',end='\t')
f13 = encode_agg_feature(trn_df,selcols=['ip','day','channel','hour'],
                         groupby=['ip','day','channel'],
                         aggregator='var'                    
                        )
print(f13.name)

print('feature 14: var of hour by (ip,app,os)', end='\t')
f14 =encode_agg_feature(trn_df,selcols=['ip','app','os','hour'], groupby=['ip','app','os'], aggregator='var')
print(f14.name)

print('feature 15: var of day by (ip,app,channel)',end='\t')
f15 = encode_agg_feature(trn_df,selcols=['ip','app','channel','day'],groupby=['ip','app','channel'], aggregator='var')
print(f15.name)

print('feature 16: mean of hour by (ip,app,channel)', end='\t')
f16 = encode_agg_feature(trn_df,selcols=['ip','app','channel','hour'],groupby=['ip','app','channel'],aggregator='mean')
print(f16.name)


trn_df[f1.name] = f1.values.astype('uint8') ; del f1;gc.collect()
trn_df[f2.name] = f2.values.astype('uint16'); del f2;gc.collect()
trn_df[f3.name] = f3.values.astype('uint8'); del f3;gc.collect()
trn_df[f4.name] = f4.values.astype('uint8'); del f4; gc.collect()
trn_df[f5.name] = f5.values.astype('uint8'); del f5; gc.collect()
trn_df[f6.name] = f6.values.astype('uint8'); del f6; gc.collect()
trn_df[f7.name] = f7.values.astype('uint8'); del f7; gc.collect()
trn_df[f8.name] = f8.values.astype('uint16'); del f8;gc.collect()
trn_df[f9.name] = f9.values.astype('uint8'); del f9;gc.collect()
trn_df[f10.name]= f10.values.astype('uint16'); del f10;gc.collect()
trn_df[f11.name]= f11.values.astype('uint16'); del f11;gc.collect()
trn_df[f12.name] = f12.values.astype('uint16'); del f12;gc.collect()

trn_df[f13.name] = f13.values.astype('float16'); del f13;gc.collect()
trn_df[f14.name] = f14.values.astype('float16'); del f14;gc.collect()
trn_df[f15.name] = f15.values.astype('float16'); del f15; gc.collect()
trn_df[f16.name] = f16.values.astype('float16'); del f16;gc.collect()

# del f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16;gc.collect()
print('complete')

feature1 : number of unique "channel" by ip	channel_nunique_ip
feature2 : cumcount of "app" by (ip,device,os)	app_cumcnt_ip_device_os
feature3 : number of unique "hour" by (ip, day)	hour_nunique_ip_day
feature4 : number of unique "app" by ip	app_nunique_ip
feature5 : number of unique "app" by (ip,os)	app_nunique_ip_os
feature6 : number of unique "device" by ip	device_nunique_ip
feature7 : number of unique "app" by channel	app_nunique_channel
feature8 : cumcount of "os" by ip 	os_cumcnt_ip
feature9 : number of unique "app" by (ip,device,os)	app_nunique_ip_device_os
feature 10 : count by (ip,day,hour)	channel_cnt_ip_day_hour
feature 11 : count by (ip,app)	channel_cnt_ip_app
feature 12 : count by (ip,app,os)	channel_cnt_ip_app_os
feature 13: var of hour by (ip,day,channel)	hour_var_ip_day_channel
feature 14: var of hour by (ip,app,os)	hour_var_ip_app_os
feature 15: var of day by (ip,app,channel)	day_var_ip_app_channel
feature 16: mean of hour by (ip,app,channel)	hour_mean_ip_app_channel


714

importanta feature 

#### next click

- next click time by ('ip','app','os','device')
- next click time 2 by ...
- cnt by next click

In [9]:
print('next click time by(ip,app,os,device)...')
gp = trn_df.groupby(['ip','app','os','device'])
print('shift 1...')
trn_df['shift_1'] = gp.click_time.shift(-1)
print('shift 2...')
trn_df['shift_2'] = gp.click_time.shift(-2)
print('timedelta to uint16...')
trn_df['next_click_dt'] = ((trn_df.shift_1 - trn_df.click_time) / np.timedelta64(1,'s')).fillna(50000).astype('uint16')
trn_df['next_click_dt2'] = ((trn_df.shift_2 - trn_df.click_time) / np.timedelta64(1,'s')).fillna(50000).astype('uint16')

print('cnt by next click...')

nextclick_cnt = trn_df.next_click_dt.value_counts().reset_index().\
    rename(columns={'index':'next_click_dt','next_click_dt':'next_click_dt_cnt'}).astype('uint32')
trn_df = trn_df.merge(nextclick_cnt,how='left')

print('gc...')
trn_df.drop(['shift_1','shift_2'],axis=1,inplace=True)
del nextclick_cnt; gc.collect()
print('complete')


next click time by(ip,app,os,device)...
shift 1...
shift 2...
timedelta to uint16...
cnt by next click...
gc...
complete


#### mean encode ON TRAINNING ONLY NOT ON TEST ###
- ip mean target
- app mean target 
- channel mean target 
- os mean target
- device mean target

NEED DATA COPY ---> may memory consuming

In [10]:
test_df = trn_df.iloc[len_trn:,]
trn_df  = trn_df.iloc[:len_trn,]

In [11]:
from sklearn.model_selection import StratifiedKFold
print('copy trn_df to make new features')
train_new = trn_df.copy()

cols = ['ip','app','channel','os','device']
for col in cols :
    train_new[col+'_mean_target'] = 0
    
y_tr = trn_df.is_attributed.values.astype(np.int8) # target 
skf = StratifiedKFold(4, random_state=0)
skf.get_n_splits(X=trn_df,y=y_tr)


for fold, (tr_idx, val_idx) in enumerate(skf.split(train_new,y_tr)):
    
    ## generate features 
    
    X_tr ,X_val = trn_df.iloc[tr_idx], trn_df.iloc[val_idx]
    
    print('fold:{} mean encoding...'.format(fold),end='\n\t')
#     print('tr_idx:{}'.format(tr_idx))
    for col in cols:        
        print(col,end='\t')
        means = X_val[col].map(X_tr.groupby(col).is_attributed.mean()) ## map mean encoding in X_tr to X_val
        X_val[col + '_mean_target'] = means.astype('float16')
        
    train_new.iloc[val_idx] = X_val
    print('')    
    del X_tr,X_val;gc.collect()
            
prior = trn_df.is_attributed.mean()
train_new[[col+'_mean_target' for col in cols]] = train_new[[col+'_mean_target' for col in cols]].fillna(prior)

trn_df = train_new
del train_new;gc.collect()
print('complete')

copy trn_df to make new features
fold:0 mean encoding...
	ip	

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


app	channel	os	device	
fold:1 mean encoding...
	ip	app	channel	os	device	
fold:2 mean encoding...
	ip	app	channel	os	device	
fold:3 mean encoding...
	ip	app	channel	os	device	


34

In [12]:
trn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000000 entries, 0 to 39999999
Data columns (total 34 columns):
app                         uint16
channel                     uint16
click_id                    float64
click_time                  datetime64[ns]
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               float64
os                          uint16
app_cumcnt_ip_device_os     uint16
os_cumcnt_ip                uint16
channel_nunique_ip          uint8
hour_nunique_ip_day         uint8
app_nunique_ip              uint8
app_nunique_ip_os           uint8
device_nunique_ip           uint8
app_nunique_channel         uint8
app_nunique_ip_device_os    uint8
channel_cnt_ip_day_hour     uint16
channel_cnt_ip_app          uint16
channel_cnt_ip_app_os       uint16
hour_var_ip_day_channel     float16
hour_var_ip_app_os          float16
day_var_ip_app_channel      float16
hour_m

___ 

#### mean encode on TEST

In [13]:
cols = ['ip','os','app','device','channel']
selcols = cols + ['is_attributed']

for col in cols:
    print('target_mean col:{}...'.format(col))
    gp = trn_df[selcols].groupby(col)
    test_df[col+'_mean_target'] = test_df[col].map(gp.is_attributed.mean()) 
    
prior = trn_df.is_attributed.mean()
test_df[[col+'_mean_target' for col in cols]] = test_df[[col+'_mean_target' for col in cols]].fillna(prior)
gc.collect()
print('complete')

target_mean col:ip...
target_mean col:os...
target_mean col:app...
target_mean col:device...
target_mean col:channel...


In [14]:
gc.collect()

91

In [19]:
test_df.shape

(18790469, 34)

In [20]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 40000000 to 58790468
Data columns (total 34 columns):
app                         uint16
channel                     uint16
click_id                    float64
click_time                  datetime64[ns]
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               float64
os                          uint16
app_cumcnt_ip_device_os     uint16
os_cumcnt_ip                uint16
channel_nunique_ip          uint8
hour_nunique_ip_day         uint8
app_nunique_ip              uint8
app_nunique_ip_os           uint8
device_nunique_ip           uint8
app_nunique_channel         uint8
app_nunique_ip_device_os    uint8
channel_cnt_ip_day_hour     uint16
channel_cnt_ip_app          uint16
channel_cnt_ip_app_os       uint16
hour_var_ip_day_channel     float16
hour_var_ip_app_os          float16
day_var_ip_app_channel      float16

___

# Save 

In [18]:
train_df.shape

(37500000, 34)

In [19]:
val_df = trn_df[(len_trn-val_size):len_trn]
train_df = trn_df[:(len_trn-val_size)]

print("train shape: ", train_df.shape)
print("valid shape: ", val_df.shape)
print("test shape : ", test_df.shape)


train shape:  (37500000, 34)
valid shape:  (2500000, 34)
test shape :  (18790469, 34)


In [21]:
print('../../feat2_{}_{}'.format(frm,to))
store = pd.HDFStore('../input/feat/feat2_{}_{}.h5'.format(frm,to))
store['train_df'] = train_df
store['valid_df'] = val_df
store['test_df'] = test_df
store.close()

../../feat2_109903890_149903890
